In [15]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import bs4 as bs4
from urllib.parse import urlparse
import requests
from collections import Counter
import pandas as pd
import re

In [16]:
df = pd.read_csv('../website_classification.csv')
df

,website_url,cleaned_website_text,Category
0,https://www.fubo.tv/lp/StreamTV/?irad=343747&i...,stream live tv sports fubotv free trial sport ...,Streaming Services
1,https://www.hulu.com/,stream tv movie live online hulu watch tv show...,Streaming Services
2,https://tv.youtube.com/,youtube tv watch dvr live sports shows news st...,Streaming Services
3,https://www.attwatchtv.com/,watchtv stream channel live tv want watchtv ex...,Streaming Services
4,https://www.klowdtv.com/,live tv streaming klowdtv klowdtv provide live...,Streaming Services
...,...,...,...
1089,https://www.slack.com,slack is your productivity platform slackslack...,Social Networking and Messaging
1090,https://www.discord.com,discord your place to talk and hang outdiscord...,Social Networking and Messaging
1091,https://www.dribbble.com,dribbble discover the worlds top designers cre...,Social Networking and Messaging
1092,https://www.producthunt.com,product hunt the best new products in tech pr...,Social Networking and Messaging


In [17]:
class ScrapTool:
    def visit_url(self, website_url):
        '''
        Visit URL. Download the Content. Initialize the BeautifulSoup object. Call parsing methods. Return Series object.
        '''
        content = requests.get(website_url, timeout=60).content
        soup = BeautifulSoup(content, 'lxml')
        result = {
            "website_url": website_url,
            "website_name": self.get_website_name(website_url),
            "website_text": self.get_html_title_tag(soup) + self.get_html_meta_tags(soup) + self.get_html_heading_tags(soup) +
                             self.get_text_content(soup)
        }

        # Convert to Series object and return
        return pd.Series(result)

    def get_website_name(self, website_url):
        '''
        Example: returns "google" from "www.google.com"
        '''
        return "".join(urlparse(website_url).netloc.split(".")[-2])

    def get_html_title_tag(self, soup):
        '''Return the text content of <title> tag from a webpage'''
        return '. '.join(soup.title.contents)

    def get_html_meta_tags(self, soup):
        '''Returns the text content of <meta> tags related to keywords and description from a webpage'''
        tags = soup.find_all(lambda tag: (tag.name == "meta") & (tag.has_attr('name') & (tag.has_attr('content'))))
        content = [str(tag["content"]) for tag in tags if tag["name"] in ['keywords', 'description']]
        return ' '.join(content)

    def get_html_heading_tags(self, soup):
        '''returns the text content of heading tags. The assumption is that headings might contain relatively important text.'''
        tags = soup.find_all(["h1", "h2", "h3", "h4", "h5", "h6"])
        content = [" ".join(tag.stripped_strings) for tag in tags]
        return ' '.join(content)

    def get_text_content(self, soup):
        '''returns the text content of the whole page with some exception to tags. See tags_to_ignore.'''
        tags_to_ignore = ['style', 'script', 'head', 'title', 'meta', '[document]', "h1", "h2", "h3", "h4", "h5", "h6", "noscript"]
        tags = soup.find_all(text=True)
        result = []
        for tag in tags:
            stripped_tag = tag.strip()
            if tag.parent.name not in tags_to_ignore \
                    and isinstance(tag, bs4.element.Comment) == False \
                    and not stripped_tag.isnumeric() \
                    and len(stripped_tag) > 0:
                result.append(stripped_tag)
        return ' '.join(result)

In [18]:
def cleaning_text(text):
    text = text.lower()
    text = text.replace('\n',' ')
    text = text.replace('\t',' ')
    text = text.replace('\r',' ')
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r' +',' ',text)
    return text

In [19]:
def content_generation(website):
    scrapTool = ScrapTool()
    try:
        web = dict(scrapTool.visit_url(website))
        text = cleaning_text(web['website_text'])
        return text
    except:
        print('Error: ',website)
        return None

In [9]:
websites = [
    "https://smartoptions.io/"
]


for website in websites:
    text = content_generation(website)
    if text is not None:
        print(text)
        # web = [website,text,'Social Networking and Messaging']
        # df.loc[len(df)] = web

 forbidden forbiddennginx


C:\Users\shrey\AppData\Local\Temp/ipykernel_5440/2194072710.py:43: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tags = soup.find_all(text=True)


In [20]:
df['Category'].value_counts()

Computers and Technology           258
Social Networking and Messaging    114
Business/Corporate                 107
Streaming Services                 103
E-Commerce                         101
Sports                             100
Games                               98
News                                93
Law and Government                  83
Adult                               21
Forums                              16
Name: Category, dtype: int64

In [14]:
df.reset_index(drop=True,inplace=True)
df.to_csv("website_classification.csv",index=False)

In [13]:
df.drop(886,inplace=True)

,website_url,cleaned_website_text,Category
886,https://smartoptions.io/,forbidden forbiddennginx,Computers and Technology
